SPARK: TIME ANALYSIS

In [2]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [5]:
import findspark
findspark
from pyspark.sql import SparkSession

#we are going to crate a SparkSession with the name "lab_spark"
spark = SparkSession.builder.appName("lab_spark").getOrCreate()

#obtain the SparkContext from the SparkSession
spark_context = spark.sparkContext

#libraries
import pandas as pd
import numpy as np
import time
#import pickle

#import functions and classes from pyspark.sql
from pyspark.sql.functions import col, round, expr, from_unixtime, unix_timestamp, date_format #module to work with columns
from pyspark.sql.types import IntegerType #module to work with integer data types

In [8]:
#we need to calculate the time it takes to read the data
start_time = time.time()

Now, we are going to read the dataset of the taxis

In [7]:
taxi_data_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").option("timestampFormat", "yyyy-MM-dd HH:mm:ss").option("mode", "PERMISSIVE").load(r"./tripdata_2017_01.csv") 
taxi_data_df.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2017-01-09 11:13:28|  2017-01-09 11:25:45|              1|          3.3|         1|                 N|         263|         161|           1|       12.5|  0.0|    0.5|       2.0|         0.0|                  0.3|        15.3|
|       1| 2017-01-09 11:32:27|  2017-01-09 11:3

In [14]:
end_time = time.time()
time_to_read=(end_time - start_time)#time it takes to read the data

In [11]:
print('Time it takes to read the data with Spark dataframes: ', time_to_read)

Time it takes to read the data with Spark dataframes:  65.47151589393616


In [ ]:
#taxi_data_df.describe().show()

Now, let's clean the data

In [22]:
#we need to calculate the time it takes to clean the data
start_time = time.time()

We are going to calculate the total duration of each trip and remove the ones that have a duration of 0

In [31]:
taxi_data_df = taxi_data_df.withColumn("trip_time_hours", round((unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600, 2)) #calculate the trip time in hours
taxi_data_df = taxi_data_df.filter(col('trip_time_hours') >= 0)

In [ ]:
#now let's count how many of them do we have

In [32]:
taxi_data_df.count()

965985

We are going to remove rows <=0 (tip_amount, total_amount, fare_amount, mta_tax)

In [41]:
#filter values of tips and other taxes lower than 0

#for column in ["tip_amount", "tolls_amount", "improvement_surcharge", "total_amount"]:
    #taxi_data_df = taxi_data_df.filter(col(column) >= 0)
#¡¡¡¡¡¡¡¡DUDA DE SI total_amount Y fare_amount SOLO DESCARTAR LOS >0 Y DEJAR LOS =0!!!!!!!!!!!!!!!!!

for column in ['tip_amount', 'mta_tax']:
    taxi_data_df = taxi_data_df.filter(taxi_data_df[column] >= 0)

In [42]:
for column in ['fare_amount', 'total_amount']:
        taxi_data_df = taxi_data_df.filter(taxi_data_df[column] > 0)

In [43]:
taxi_data_df.count()

965794